In [1]:
import anndata as ad
import matplotlib.pyplot as plt
import mudata as md
import muon
import scanpy as sc
from scvi.data import pbmc_seurat_v4_cite_seq
from scvi.model import TOTALVI
import sys
import numpy as np
import pickle
from copy import deepcopy
import numpy as np
import pandas as pd

import cosg as cosg
import importlib
import time
importlib.reload(cosg)

/home/ubartu/miniconda3/envs/hap/lib/python3.9/site-packages/scvi/_settings.py:63: UserWarning: Since v1.0.0, scvi-tools no longer uses a random seed by default. Run `scvi.settings.seed = 0` to reproduce results from previous versions.
  self.seed = seed
/home/ubartu/miniconda3/envs/hap/lib/python3.9/site-packages/scvi/_settings.py:70: UserWarning: Setting `dl_pin_memory_gpu_training` is deprecated in v1.0 and will be removed in v1.1. Please pass in `pin_memory` to the data loaders instead.
  self.dl_pin_memory_gpu_training = (


<module 'cosg' from '/home/ubartu/miniconda3/envs/hap/lib/python3.9/site-packages/cosg/__init__.py'>

In [2]:
dataLoc = './data/'
adata = pbmc_seurat_v4_cite_seq(save_path=dataLoc, apply_filters=True, aggregate_proteins=True, mask_protein_batches=0)

adata.layers["counts"] = adata.X.copy()
# sc.pp.normalize_total(adata)
# sc.pp.log1p(adata)
adata.obs_names_make_unique()

INFO     File ./data/pbmc_seurat_v4.h5ad already downloaded                                                        


In [3]:
protein_adata = ad.AnnData(adata.obsm["protein_counts"])
protein_adata.obs_names = adata.obs_names
del adata.obsm["protein_counts"]
# mdata = md.MuData({"rna": adata, "protein": protein_adata})
# mdata.var_names_make_unique()

In [4]:
np.unique(adata.obs['celltype.l2'])

array(['ASDC', 'B intermediate', 'B memory', 'B naive', 'CD14 Mono',
       'CD16 Mono', 'CD4 CTL', 'CD4 Naive', 'CD4 Proliferating',
       'CD4 TCM', 'CD4 TEM', 'CD8 Naive', 'CD8 Proliferating', 'CD8 TCM',
       'CD8 TEM', 'Eryth', 'HSPC', 'ILC', 'MAIT', 'NK',
       'NK Proliferating', 'NK_CD56bright', 'Plasmablast', 'Platelet',
       'Treg', 'cDC1', 'cDC2', 'dnT', 'gdT', 'pDC'], dtype=object)

In [5]:
print(len(np.unique(adata.obs['celltype.l2'])))

30


In [6]:
help(cosg.cosg)

Help on function cosg in module cosg.cosg:

cosg(adata, groupby='CellTypes', groups: Union[Literal['all'], Iterable[str]] = 'all', mu=1, remove_lowly_expressed: bool = False, expressed_pct: Optional[float] = 0.1, n_genes_user: int = 50, key_added: Optional[str] = None, use_raw: bool = True, layer: Optional[str] = None, reference: str = 'rest', copy: bool = False)
    Marker gene identification for single-cell sequencing data using COSG.
    
    Parameters
    ----------
    adata
        Annotated data matrix. Note: input paramters are simliar to the parameters used for scanpy's rank_genes_groups() function.
    groupby
        The key of the cell groups in .obs, the default value is set to 'CellTypes'.
    groups
        Subset of cell groups, e.g. [`'g1'`, `'g2'`, `'g3'`], to which comparison shall be restricted. The default value is 'all', and all groups will be compared.
    mu
        The penalty restricting marker genes expressing in non-target cell groups. Larger value represen

In [7]:
adata

AnnData object with n_obs × n_vars = 152094 × 20729
    obs: 'nCount_ADT', 'nFeature_ADT', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'lane', 'donor', 'time', 'celltype.l1', 'celltype.l2', 'celltype.l3', 'Phase', 'nCount_SCT', 'nFeature_SCT', 'X_index', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'Protein log library size', 'Number proteins detected', 'RNA log library size'
    var: 'mt'
    layers: 'counts'

In [8]:
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

In [9]:
t0= time.time()
cosg.cosg(adata,
    key_added='cosg',
        mu=1,
        n_genes_user=30,
        groupby='celltype.l2')
runtime_cosg = time.time() - t0
print(runtime_cosg)

**finished identifying marker genes by COSG**
4.292004585266113


In [11]:
marker_gene=pd.DataFrame(adata.uns['cosg']['names'])
marker_gene

,ASDC,B intermediate,B memory,B naive,CD14 Mono,CD16 Mono,CD4 CTL,CD4 Naive,CD4 Proliferating,CD4 TCM,...,NK Proliferating,NK_CD56bright,Plasmablast,Platelet,Treg,cDC1,cDC2,dnT,gdT,pDC
0,AXL,LINC01857,LINC01781,TCL1A,CD14,CDKN1C,TRBV18,FHIT,CCNB2,AQP3,...,TYMS,SPTSSB,TNFRSF17,TUBB1,FOXP3,CLEC9A,FCER1A,PTPN3,TRDV2,CLEC4C
1,SIGLEC6,SOX5,SSPN,IGHD,VCAN,HES4,GFPT2,TSHZ2,DLGAP5,TNFRSF4,...,CLSPN,XCL1,TXNDC5,CAVIN2,RTKN2,XCR1,ENHO,FXYD2,TRDV1,LILRA4
2,ATP1A2,GPM6A,COCH,FCER2,S100A12,CKB,TRBV10-2,ADTRP,PLK1,NEFL,...,TK1,TNFRSF11A,IGF1,GNG11,LINC01281,IDO1,CLEC10A,AC079341.1,TRGC1,LRRC26
3,HAMP,KLK1,TEX9,IGHM,CSF3R,MS4A4A,TRBV5-6,CCR7,CDC20,AP3M2,...,DTL,KIR2DL4,GLDC,GP9,FANK1,PPY,PKIB,MIR4422HG,TRDC,SERPINF1
4,PLS3,FCRL2,SHISA8,FCRL1,S100A8,CTSL,TRAV38-1,TRABD2A,HMMR,TTC39C-AS1,...,CDC45,PPP1R9A,SDC1,PF4,IL2RA,ENPP1,CD1C,OGN,TRGV9,SCT
5,CDH1,TNFRSF13B,AIM2,CD22,MS4A6A,TCF7L2,LINC01793,LEF1,CEP55,INPP4B,...,MCM10,IGFBP4,SPACA3,SPARC,ARG1,RAB7B,CD1E,GDF10,TRGC2,EPHB1
6,CTSV,GALNTL6,BTNL9,PLPP5,MNDA,C1QA,PROK2,ANKRD55,CENPA,CD40LG,...,CKAP2L,ADGRG3,AC106897.1,PPBP,AC133644.2,CLNK,MRC1,AC004585.1,TRDV3,LINC00996
7,VASN,CLECL1,IGHG3,COL19A1,FCN1,CASP5,TRAV17,CHRM3-AS2,UBE2C,ST6GALNAC1,...,MKI67,CDHR1,GPRC5D,TREML1,RBMS3,SEPT3,CLIC2,AC006058.1,TRGV5,LAMP5
8,CCDC188,AC104024.1,IGHG1,PCDH9,CD36,VMO1,KIF19,IL6ST,KIF20A,CCR4,...,PCLAF,ZMAT4,MIXL1,CMTM5,CTLA4,LINC02206,IL1R2,CAV1,CYP24A1,SMPD3
9,BICC1,FCRLA,IGHA2,LINC00926,CD93,LYPD2,GZMH,MAL,CDKN3,IL7R,...,PKMYT1,XCL2,SCNN1B,CLU,GCNT4,AL390729.1,NDRG2,DSCAML1,KLRG1,SCAMP5


In [12]:
marker_gene.to_csv('./data/cosg_30_markergenes.csv')

In [11]:
marker_gene_names = marker_gene.values.flatten().tolist()
marker_gene_names = np.array(marker_gene_names, dtype=str)
marker_gene_names.shape

(900,)